# This is the Jupyter Notebook for the MAGICODE project

### First install some modules that are (usually) not installed by default

In [1]:
!pip install tensorflow
# install sklearn to use train_test_split function
!pip install sklearn
# install opencv to use cv2 module in get_preprocessed_img
!pip install opencv-python

You should consider upgrading via the '/home/va281/projects/python-stuff/jupyterenvironment/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/va281/projects/python-stuff/jupyterenvironment/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/va281/projects/python-stuff/jupyterenvironment/bin/python -m pip install --upgrade pip' command.


### Then import some libraries and modules that are needed for the code to run

In [2]:
import os
import sys
import sklearn.model_selection as model_selection
import glob
import shutil
import numpy as np
from pathlib import Path
from os.path import join

### Unzip the dataset

In [3]:
# join the volumes together into a single zip file
!zip -s 0 dataset.zip -O dataset_joined.zip
# unzip the newly assembled archive into the current folder
!unzip dataset_joined.zip -d ./

if os.listdir('dataset') :
    print ('files unzipped')

 copying: dataset/
 copying: dataset/000-jedka-6433-whdzcxpz.gui
 copying: dataset/000-jedka-6433-whdzcxpz.png
 copying: dataset/000-juycn-5502-fyfqniup.gui
 copying: dataset/000-juycn-5502-fyfqniup.png
 copying: dataset/000-lvjhh-4287-xnxymgmj.gui
 copying: dataset/000-lvjhh-4287-xnxymgmj.png
 copying: dataset/001-mvyrc-8921-pihmvgjv.gui
 copying: dataset/001-mvyrc-8921-pihmvgjv.png
 copying: dataset/001-qwesp-0166-ebgkfuec.gui
 copying: dataset/001-qwesp-0166-ebgkfuec.png
 copying: dataset/002-ndluf-8876-efimxruq.gui
 copying: dataset/002-ndluf-8876-efimxruq.png
 copying: dataset/002-qetqg-0385-etbfymjy.gui
 copying: dataset/002-qetqg-0385-etbfymjy.png
 copying: dataset/003-oesgn-3859-rhhylzmk.gui
 copying: dataset/003-oesgn-3859-rhhylzmk.png
 copying: dataset/004-anxol-3624-demqcwan.gui
 copying: dataset/004-anxol-3624-demqcwan.png
 copying: dataset/007-kxblk-7571-ehpiiupq.gui
 copying: dataset/007-kxblk-7571-ehpiiupq.png
 copying: dataset/007-tqijd-7598-ebpzzkwh.gui
 copying: datas

 copying: dataset/351-qcqhx-0996-ctcszrwn.png
 copying: dataset/351-wmvau-7357-iuwtjvrq.gui
 copying: dataset/351-wmvau-7357-iuwtjvrq.png
 copying: dataset/351-yqmiy-3752-wfeuxznm.gui
 copying: dataset/351-yqmiy-3752-wfeuxznm.png
 copying: dataset/352-fsvsm-4102-ndiorgna.gui
 copying: dataset/352-fsvsm-4102-ndiorgna.png
 copying: dataset/352-rivnz-8398-mfaufwas.gui
 copying: dataset/352-rivnz-8398-mfaufwas.png
 copying: dataset/353-bktqw-6321-heatfgbv.gui
 copying: dataset/353-bktqw-6321-heatfgbv.png
 copying: dataset/354-awitj-7445-ecpjydvg.gui
 copying: dataset/354-awitj-7445-ecpjydvg.png
 copying: dataset/354-efjhb-3954-wxgtcjvl.gui
 copying: dataset/354-efjhb-3954-wxgtcjvl.png
 copying: dataset/354-qpfgv-9655-qdztglfg.gui
 copying: dataset/354-qpfgv-9655-qdztglfg.png
 copying: dataset/354-sntll-8208-jyftmcvn.gui
 copying: dataset/354-sntll-8208-jyftmcvn.png
 copying: dataset/355-lorsn-6315-ypmmivfj.gui
 copying: dataset/355-lorsn-6315-ypmmivfj.png
 copying: dataset/356-fvanh-3150-y

 copying: dataset/813-jxfdi-2581-qlvhkbdo.png
 copying: dataset/813-maupx-7446-vpbbwrag.gui
 copying: dataset/813-maupx-7446-vpbbwrag.png
 copying: dataset/813-vwkdj-6181-dtekhciw.gui
 copying: dataset/813-vwkdj-6181-dtekhciw.png
 copying: dataset/815-jbqzj-0481-tjwlqamr.gui
 copying: dataset/815-jbqzj-0481-tjwlqamr.png
 copying: dataset/815-jdrib-1842-vuqydyjd.gui
 copying: dataset/815-jdrib-1842-vuqydyjd.png
 copying: dataset/815-tyzjq-3712-kyaxwpnr.gui
 copying: dataset/815-tyzjq-3712-kyaxwpnr.png
 copying: dataset/816-akkmy-4926-mlaihptl.gui
 copying: dataset/816-akkmy-4926-mlaihptl.png
 copying: dataset/816-reyqe-1644-mxsnfeuk.gui
 copying: dataset/816-reyqe-1644-mxsnfeuk.png
 copying: dataset/816-rxarz-8378-jbujctdj.gui
 copying: dataset/816-rxarz-8378-jbujctdj.png
 copying: dataset/817-pfjlq-3178-yhaqfosg.gui
 copying: dataset/817-pfjlq-3178-yhaqfosg.png
 copying: dataset/818-qrymy-1515-pcdmewyh.gui
 copying: dataset/818-qrymy-1515-pcdmewyh.png
 copying: dataset/818-uocgz-1489-i

Archive:  dataset_joined.zip
   creating: ./dataset/
  inflating: ./dataset/000-jedka-6433-whdzcxpz.gui  
  inflating: ./dataset/000-jedka-6433-whdzcxpz.png  
  inflating: ./dataset/000-juycn-5502-fyfqniup.gui  
  inflating: ./dataset/000-juycn-5502-fyfqniup.png  
  inflating: ./dataset/000-lvjhh-4287-xnxymgmj.gui  
  inflating: ./dataset/000-lvjhh-4287-xnxymgmj.png  
  inflating: ./dataset/001-mvyrc-8921-pihmvgjv.gui  
  inflating: ./dataset/001-mvyrc-8921-pihmvgjv.png  
  inflating: ./dataset/001-qwesp-0166-ebgkfuec.gui  
  inflating: ./dataset/001-qwesp-0166-ebgkfuec.png  
  inflating: ./dataset/002-ndluf-8876-efimxruq.gui  
  inflating: ./dataset/002-ndluf-8876-efimxruq.png  
  inflating: ./dataset/002-qetqg-0385-etbfymjy.gui  
  inflating: ./dataset/002-qetqg-0385-etbfymjy.png  
  inflating: ./dataset/003-oesgn-3859-rhhylzmk.gui  
  inflating: ./dataset/003-oesgn-3859-rhhylzmk.png  
  inflating: ./dataset/004-anxol-3624-demqcwan.gui  
  inflating: ./dataset/004-anxol-3624-demqcwan

  inflating: ./dataset/053-xtxme-8735-buyixdew.png  
  inflating: ./dataset/054-dovoq-0958-uvcmdmzt.gui  
  inflating: ./dataset/054-dovoq-0958-uvcmdmzt.png  
  inflating: ./dataset/054-kbqkh-8865-vrjqjeoj.gui  
  inflating: ./dataset/054-kbqkh-8865-vrjqjeoj.png  
  inflating: ./dataset/054-tsuyb-2778-ogvywwsa.gui  
  inflating: ./dataset/054-tsuyb-2778-ogvywwsa.png  
  inflating: ./dataset/055-fkzzi-7491-tzgrmsou.gui  
  inflating: ./dataset/055-fkzzi-7491-tzgrmsou.png  
  inflating: ./dataset/055-plixz-0703-tpxrpfzl.gui  
  inflating: ./dataset/055-plixz-0703-tpxrpfzl.png  
  inflating: ./dataset/056-cfnbp-0312-ctixviwr.gui  
  inflating: ./dataset/056-cfnbp-0312-ctixviwr.png  
  inflating: ./dataset/056-oagnt-3121-flxqxenz.gui  
  inflating: ./dataset/056-oagnt-3121-flxqxenz.png  
  inflating: ./dataset/056-pfphu-8398-spccyokc.gui  
  inflating: ./dataset/056-pfphu-8398-spccyokc.png  
  inflating: ./dataset/056-qjpya-8823-toxgbqmg.gui  
  inflating: ./dataset/056-qjpya-8823-toxgbqmg

...

  inflating: ./dataset/671-gwour-2181-majpkjux.png  
  inflating: ./dataset/671-iwzet-2713-cmiyzzhj.gui  
  inflating: ./dataset/671-iwzet-2713-cmiyzzhj.png  
  inflating: ./dataset/671-lmtvb-1723-vfvgqyic.gui  
  inflating: ./dataset/671-lmtvb-1723-vfvgqyic.png  
  inflating: ./dataset/671-taiww-8500-nbexwwov.gui  
  inflating: ./dataset/671-taiww-8500-nbexwwov.png  
  inflating: ./dataset/671-vphbn-5802-knzvfvuz.gui  
  inflating: ./dataset/671-vphbn-5802-knzvfvuz.png  
  inflating: ./dataset/673-ffcjt-8315-dzpvhlwu.gui  
  inflating: ./dataset/673-ffcjt-8315-dzpvhlwu.png  
  inflating: ./dataset/673-riqjx-8868-whwijqrn.gui  
  inflating: ./dataset/673-riqjx-8868-whwijqrn.png  
  inflating: ./dataset/673-wtbui-3848-sfkhioem.gui  
  inflating: ./dataset/673-wtbui-3848-sfkhioem.png  
  inflating: ./dataset/673-zrhwo-0920-awvtosfn.gui  
  inflating: ./dataset/673-zrhwo-0920-awvtosfn.png  
  inflating: ./dataset/674-cnaor-3175-rklvwkce.gui  
  inflating: ./dataset/674-cnaor-3175-rklvwkce

  inflating: ./dataset/755-bfpkj-2481-omqjsstj.png  
  inflating: ./dataset/755-vtxye-4542-mcwpbiti.gui  
  inflating: ./dataset/755-vtxye-4542-mcwpbiti.png  
  inflating: ./dataset/755-ymqmo-6972-qaeukvye.gui  
  inflating: ./dataset/755-ymqmo-6972-qaeukvye.png  
  inflating: ./dataset/756-ncxss-6516-bcegouyz.gui  
  inflating: ./dataset/756-ncxss-6516-bcegouyz.png  
  inflating: ./dataset/756-pwhwd-5946-ztpnedrf.gui  
  inflating: ./dataset/756-pwhwd-5946-ztpnedrf.png  
  inflating: ./dataset/756-svmyr-4148-hffikaia.gui  
  inflating: ./dataset/756-svmyr-4148-hffikaia.png  
  inflating: ./dataset/758-xhjbd-5925-irruwbbb.gui  
  inflating: ./dataset/758-xhjbd-5925-irruwbbb.png  
  inflating: ./dataset/759-dguzq-6919-sfrpolsl.gui  
  inflating: ./dataset/759-dguzq-6919-sfrpolsl.png  
  inflating: ./dataset/759-ijieh-2478-ttkaliwe.gui  
  inflating: ./dataset/759-ijieh-2478-ttkaliwe.png  
  inflating: ./dataset/764-lgmvd-3493-yvbgxwmw.gui  
  inflating: ./dataset/764-lgmvd-3493-yvbgxwmw

  inflating: ./dataset/846-ikuij-7180-uzeycuek.png  
  inflating: ./dataset/846-llwxd-7372-omiflfih.gui  
  inflating: ./dataset/846-llwxd-7372-omiflfih.png  
  inflating: ./dataset/846-rdaed-6341-vfstevun.gui  
  inflating: ./dataset/846-rdaed-6341-vfstevun.png  
  inflating: ./dataset/847-fmcog-6099-aebiwtll.gui  
  inflating: ./dataset/847-fmcog-6099-aebiwtll.png  
  inflating: ./dataset/847-gmxdl-2481-htdlcczz.gui  
  inflating: ./dataset/847-gmxdl-2481-htdlcczz.png  
  inflating: ./dataset/848-kfogi-8740-jjtlzzqt.gui  
  inflating: ./dataset/848-kfogi-8740-jjtlzzqt.png  
  inflating: ./dataset/848-xjvud-4891-mtqxeeno.gui  
  inflating: ./dataset/848-xjvud-4891-mtqxeeno.png  
  inflating: ./dataset/849-twicd-8242-elmbapun.gui  
  inflating: ./dataset/849-twicd-8242-elmbapun.png  
  inflating: ./dataset/850-czihu-6566-urrkenth.gui  
  inflating: ./dataset/850-czihu-6566-urrkenth.png  
  inflating: ./dataset/850-jgosb-9650-vuxegzdc.gui  
  inflating: ./dataset/850-jgosb-9650-vuxegzdc

  inflating: ./dataset/936-vonbw-5680-cqhetajg.png  
  inflating: ./dataset/936-zcgqu-5660-ecpiglin.gui  
  inflating: ./dataset/936-zcgqu-5660-ecpiglin.png  
  inflating: ./dataset/937-eegzw-2804-xnzvvslq.gui  
  inflating: ./dataset/937-eegzw-2804-xnzvvslq.png  
  inflating: ./dataset/937-nekec-2654-ewmcjckf.gui  
  inflating: ./dataset/937-nekec-2654-ewmcjckf.png  
  inflating: ./dataset/937-ylscc-9428-mpazdzpj.gui  
  inflating: ./dataset/937-ylscc-9428-mpazdzpj.png  
  inflating: ./dataset/937-yqzjl-2172-mviaccht.gui  
  inflating: ./dataset/937-yqzjl-2172-mviaccht.png  
  inflating: ./dataset/939-sfilw-9428-afahjphq.gui  
  inflating: ./dataset/939-sfilw-9428-afahjphq.png  
  inflating: ./dataset/939-shmor-4875-jgqhvtgr.gui  
  inflating: ./dataset/939-shmor-4875-jgqhvtgr.png  
  inflating: ./dataset/940-iqpud-2303-ojtuydje.gui  
  inflating: ./dataset/940-iqpud-2303-ojtuydje.png  
  inflating: ./dataset/940-nwznn-7125-lryoshdq.gui  
  inflating: ./dataset/940-nwznn-7125-lryoshdq

  inflating: ./dataset/973-nbskh-3291-qmzfihda.png  
  inflating: ./dataset/973-qusfo-9613-flkusnhj.gui  
  inflating: ./dataset/973-qusfo-9613-flkusnhj.png  
  inflating: ./dataset/974-afozr-8106-bjwmhtpa.gui  
  inflating: ./dataset/974-afozr-8106-bjwmhtpa.png  
  inflating: ./dataset/974-qyffz-7715-xemhvgvt.gui  
  inflating: ./dataset/974-qyffz-7715-xemhvgvt.png  
  inflating: ./dataset/976-mmfok-8936-kxpubvaj.gui  
  inflating: ./dataset/976-mmfok-8936-kxpubvaj.png  
  inflating: ./dataset/976-nlxxj-5239-xlefxysx.gui  
  inflating: ./dataset/976-nlxxj-5239-xlefxysx.png  
  inflating: ./dataset/976-qaihs-8400-zusjsbqz.gui  
  inflating: ./dataset/976-qaihs-8400-zusjsbqz.png  
  inflating: ./dataset/976-sekcz-0444-emfsyfhx.gui  
  inflating: ./dataset/976-sekcz-0444-emfsyfhx.png  
  inflating: ./dataset/977-eqjiz-1710-duuumiat.gui  
  inflating: ./dataset/977-eqjiz-1710-duuumiat.png  
  inflating: ./dataset/978-ehnde-0930-zbbddwnj.gui  
  inflating: ./dataset/978-ehnde-0930-zbbddwnj

### Define some values used later

In [4]:
TRAINING_SET_NAME = 'training_set'
EVALUATION_SET_NAME = 'eval_set'
IMAGE_SIZE = 256
BATCH_SIZE = 64
CONTEXT_LENGTH = 48
START_TOKEN = "<START>"
END_TOKEN = "<END>"
PLACEHOLDER = " "
SEPARATOR = '->'
EPOCHS = 10

### Split dataset into training and evaluation sets

In [5]:
# define source folder
source = 'dataset'
# get all file paths
all_files = os.listdir(source)
# build a generic image path (e.g. 'all_data/dataset/*.png')
images_path = join(source, '*.png')
# get all images paths
img_files = glob.glob(images_path)
# remove files extension from files paths
img_files_without_extension = [Path(img_file).stem for img_file in img_files]

# training set will be 6 times the size of the evaluation set
distribution = 6

# splits randomly the files into two sets
train_set,eval_set = model_selection.train_test_split(img_files_without_extension, train_size=(distribution / 10))

# create the TRAINING_SET_NAME and EVALUATION_SET_NAME directories if they do not exist
if not os.path.exists(join(source, TRAINING_SET_NAME)):
    os.makedirs(join(source, TRAINING_SET_NAME))
if not os.path.exists(join(source, EVALUATION_SET_NAME)):
    os.makedirs(join(source, EVALUATION_SET_NAME))

# copy the files (img and gui) from the all_data folder into the training_set folder
for file in train_set:
    shutil.copyfile(join(source, file + '.png'), join(source, TRAINING_SET_NAME, file + '.png'))
    shutil.copyfile(join(source, file + '.gui'), join(source, TRAINING_SET_NAME, file + '.gui'))

# copy the files (img and gui) from the all_data folder into the eval_set folder
for file in eval_set:
    shutil.copyfile(join(source, file + '.png'), join(source, EVALUATION_SET_NAME, file + '.png'))
    shutil.copyfile(join(source, file + '.gui'), join(source, EVALUATION_SET_NAME, file + '.gui'))

print('Training dataset: {}'.format(join(source, TRAINING_SET_NAME)))
print('Evaluation dataset: {}'.format(join(source, EVALUATION_SET_NAME)))


Training dataset: dataset/training_set
Evaluation dataset: dataset/eval_set


### Define some Classes and functions that will be used a few times

In [6]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! REFACTOR - Remove unused functions, use .join etc.
class Utils:
    @staticmethod
    def sparsify(label_vector, output_size):
        sparse_vector = []

        for label in label_vector:
            sparse_label = np.zeros(output_size)
            sparse_label[label] = 1

            sparse_vector.append(sparse_label)

        return np.array(sparse_vector)

    @staticmethod
    def get_preprocessed_img(img_path, image_size):
        import cv2
        img = cv2.imread(img_path)
        if not img is None:
            img = cv2.resize(img, (image_size, image_size))
            img = img.astype('float32')
            img /= 255
        return img
    
class Sampler:
    def __init__(self, voc_path, input_shape, output_size, context_length):
        self.voc = Vocabulary()
        self.voc.retrieve(voc_path)

        self.input_shape = input_shape
        self.output_size = output_size

        print('Vocabulary size: {}'.format(self.voc.size))
        print('Input shape: {}'.format(self.input_shape))
        print('Output size: {}'.format(self.output_size))

        self.context_length = context_length

    def predict_greedy(self, model, input_img, require_sparse_label=True, sequence_length=150, verbose=False):
        current_context = [self.voc.vocabulary[PLACEHOLDER]] * (self.context_length - 1)
        current_context.append(self.voc.vocabulary[START_TOKEN])
        if require_sparse_label:
            current_context = Utils.sparsify(current_context, self.output_size)

        predictions = START_TOKEN
        out_probas = []

        for i in range(0, sequence_length):
            if verbose:
                print('predicting {}/{}...'.format(i, sequence_length))

            probas = model.predict(input_img, np.array([current_context]))
            prediction = np.argmax(probas)
            out_probas.append(probas)

            new_context = []
            for j in range(1, self.context_length):
                new_context.append(current_context[j])

            if require_sparse_label:
                sparse_label = np.zeros(self.output_size)
                sparse_label[prediction] = 1
                new_context.append(sparse_label)
            else:
                new_context.append(prediction)

            current_context = new_context

            predictions += self.voc.token_lookup[prediction]

            if self.voc.token_lookup[prediction] == END_TOKEN:
                break

        return predictions, out_probas


class Vocabulary:
    def __init__(self):
        self.binary_vocabulary = {}
        self.vocabulary = {}
        self.token_lookup = {}
        self.size = 0

        self.append(START_TOKEN)
        self.append(END_TOKEN)
        self.append(PLACEHOLDER)

    def append(self, token):
        if token not in self.vocabulary:
            self.vocabulary[token] = self.size
            self.token_lookup[self.size] = token
            self.size += 1

    def create_binary_representation(self):
        if sys.version_info >= (3,):
            items = self.vocabulary.items()
        else:
            items = self.vocabulary.iteritems()
        for key, value in items:
            binary = np.zeros(self.size)
            binary[value] = 1
            self.binary_vocabulary[key] = binary

    def get_serialized_binary_representation(self):
        if len(self.binary_vocabulary) == 0:
            self.create_binary_representation()

        string = ''
        if sys.version_info >= (3,):
            items = self.binary_vocabulary.items()
        else:
            items = self.binary_vocabulary.iteritems()
        for key, value in items:
            array_as_string = np.array2string(value, separator=',', max_line_width=self.size * self.size)
            string += '{}{}{}\n'.format(key, SEPARATOR, array_as_string[1:len(array_as_string) - 1])
        return string

    def save(self, path):
        output_file_name = '{}/words.vocab'.format(path)
        output_file = open(output_file_name, 'w')
        output_file.write(self.get_serialized_binary_representation())
        output_file.close()

    def retrieve(self, path):
        input_file = open('{}/words.vocab'.format(path), 'r')
        buffer = ''
        for line in input_file:
            try:
                separator_position = len(buffer) + line.index(SEPARATOR)
                buffer += line
                key = buffer[:separator_position]
                value = buffer[separator_position + len(SEPARATOR):]
                value = np.fromstring(value, sep=',')

                self.binary_vocabulary[key] = value
                self.vocabulary[key] = np.where(value == 1)[0][0]
                self.token_lookup[np.where(value == 1)[0][0]] = key

                buffer = ''
            except ValueError:
                buffer += line
        input_file.close()
        self.size = len(self.vocabulary)

class Generator:
    @staticmethod
    def data_generator(voc, gui_paths, img_paths, batch_size, generate_binary_sequences=False, verbose=False, loop_only_one=False):
        assert len(gui_paths) == len(img_paths)
        voc.create_binary_representation()

        while 1:
            batch_input_images = []
            batch_partial_sequences = []
            batch_next_words = []
            sample_in_batch_counter = 0

            for i in range(0, len(gui_paths)):
                if img_paths[i].find('.png') != -1:
                    img = Utils.get_preprocessed_img(img_paths[i], IMAGE_SIZE)
                else:
                    img = np.load(img_paths[i])['features']
                gui = open(gui_paths[i], 'r')

                token_sequence = [START_TOKEN]
                for line in gui:
                    line = line.replace(',', ' ,').replace('\n', ' \n')
                    tokens = line.split(' ')
                    for token in tokens:
                        voc.append(token)
                        token_sequence.append(token)
                token_sequence.append(END_TOKEN)

                suffix = [PLACEHOLDER] * CONTEXT_LENGTH

                a = np.concatenate([suffix, token_sequence])
                for j in range(0, len(a) - CONTEXT_LENGTH):
                    context = a[j:j + CONTEXT_LENGTH]
                    label = a[j + CONTEXT_LENGTH]

                    batch_input_images.append(img)
                    batch_partial_sequences.append(context)
                    batch_next_words.append(label)
                    sample_in_batch_counter += 1

                    if sample_in_batch_counter == batch_size or (loop_only_one and i == len(gui_paths) - 1):
                        if verbose:
                            print('Generating sparse vectors...')
                        batch_next_words = Dataset.sparsify_labels(batch_next_words, voc)
                        if generate_binary_sequences:
                            batch_partial_sequences = Dataset.binarize(batch_partial_sequences, voc)
                        else:
                            batch_partial_sequences = Dataset.indexify(batch_partial_sequences, voc)

                        if verbose:
                            print('Convert arrays...')
                        batch_input_images = np.array(batch_input_images)
                        batch_partial_sequences = np.array(batch_partial_sequences)
                        batch_next_words = np.array(batch_next_words)

                        if verbose:
                            print('Yield batch')
                        yield ([batch_input_images, batch_partial_sequences], batch_next_words)

                        batch_input_images = []
                        batch_partial_sequences = []
                        batch_next_words = []
                        sample_in_batch_counter = 0
                        
class Dataset:
    def __init__(self):
        self.input_shape = None
        self.output_size = None

        self.ids = []
        self.input_images = []
        self.partial_sequences = []
        self.next_words = []

        self.voc = Vocabulary()
        self.size = 0

    @staticmethod
    def load_paths_only(path):
        print('Parsing data...')
        gui_paths = []
        img_paths = []
        for f in os.listdir(path):
            if f.find('.gui') != -1:
                path_gui = join(path, f)
                gui_paths.append(path_gui)
                file_name = f[:f.find('.gui')]

                if os.path.isfile(join(path, file_name + '.png')):
                    path_img = format(path, file_name + '.png')
                    img_paths.append(path_img)
                elif os.path.isfile(join(path, file_name + '.npz')):
                    path_img = join(path, file_name + '.npz')
                    img_paths.append(path_img)

        assert len(gui_paths) == len(img_paths)
        return gui_paths, img_paths

    def load(self, path, generate_binary_sequences=False):
        print('Loading data...')
        for f in os.listdir(path):
            if f.find('.gui') != -1:
                gui = open(join(path, f), 'r')
                file_name = Path(f).stem
                if os.path.isfile(join(path, file_name + '.png')):
                    img = Utils.get_preprocessed_img(join(path, file_name + '.png'), IMAGE_SIZE)
                    self.append(file_name, gui, img)
                elif os.path.isfile(join(path, file_name + '.npz')):
                    img = np.load(join(path, file_name + '.npz'))['features']
                    self.append(file_name, gui, img)

        print('Generating sparse vectors...')
        self.voc.create_binary_representation()
        self.next_words = self.sparsify_labels(self.next_words, self.voc)
        if generate_binary_sequences:
            self.partial_sequences = self.binarize(self.partial_sequences, self.voc)
        else:
            self.partial_sequences = self.indexify(self.partial_sequences, self.voc)

        self.size = len(self.ids)
        assert self.size == len(self.input_images) == len(self.partial_sequences) == len(self.next_words)
        assert self.voc.size == len(self.voc.vocabulary)

        print('Dataset size: {}'.format(self.size))
        print('Vocabulary size: {}'.format(self.voc.size))

        self.input_shape = self.input_images[0].shape
        self.output_size = self.voc.size

        print('Input shape: {}'.format(self.input_shape))
        print('Output size: {}'.format(self.output_size))

    def convert_arrays(self):
        print('Convert arrays...')
        self.input_images = np.array(self.input_images)
        self.partial_sequences = np.array(self.partial_sequences)
        self.next_words = np.array(self.next_words)

    def append(self, sample_id, gui, img, to_show=False):
        if to_show:
            pic = img * 255
            pic = np.array(pic, dtype=np.uint8)
            Utils.show(pic)

        token_sequence = [START_TOKEN]
        for line in gui:
            line = line.replace(',', ' ,').replace('\n', ' \n')
            tokens = line.split(' ')
            for token in tokens:
                self.voc.append(token)
                token_sequence.append(token)
        token_sequence.append(END_TOKEN)

        suffix = [PLACEHOLDER] * CONTEXT_LENGTH

        a = np.concatenate([suffix, token_sequence])
        for j in range(0, len(a) - CONTEXT_LENGTH):
            context = a[j:j + CONTEXT_LENGTH]
            label = a[j + CONTEXT_LENGTH]

            self.ids.append(sample_id)
            self.input_images.append(img)
            self.partial_sequences.append(context)
            self.next_words.append(label)

    @staticmethod
    def indexify(partial_sequences, voc):
        temp = []
        for sequence in partial_sequences:
            sparse_vectors_sequence = []
            for token in sequence:
                sparse_vectors_sequence.append(voc.vocabulary[token])
            temp.append(np.array(sparse_vectors_sequence))

        return temp

    @staticmethod
    def binarize(partial_sequences, voc):
        temp = []
        for sequence in partial_sequences:
            sparse_vectors_sequence = []
            for token in sequence:
                sparse_vectors_sequence.append(voc.binary_vocabulary[token])
            temp.append(np.array(sparse_vectors_sequence))

        return temp

    @staticmethod
    def sparsify_labels(next_words, voc):
        temp = []
        for label in next_words:
            temp.append(voc.binary_vocabulary[label])

        return temp

    def save_metadata(self, path):
        np.save(join(path, 'meta_dataset'), np.array([self.input_shape, self.output_size, self.size], dtype=object), allow_pickle=True)


### Transform training set into numpy arrays

In [7]:
#define source and destination folders
source = join('dataset', 'training_set')
destination = join('dataset', 'training_features')

# create the training_features directory if it does not exist
if not os.path.exists(destination):
    os.makedirs(destination)

# transform images in training dataset (i.e. normalized pixel values and resized pictures) to numpy arrays (smaller files, useful if uploading the set to train a model in the cloud)
for f in os.listdir(source):
    if f.find('.png') != -1:
        img = Utils.get_preprocessed_img(join(source, f), IMAGE_SIZE)
        file_name = f[:f.find('.png')]

        np.savez_compressed(join(destination, file_name), features=img)
        retrieve = np.load(join(destination, file_name + '.npz'))['features']
        
        assert np.array_equal(img, retrieve)
        
        shutil.copyfile(join(source, file_name + '.gui'), join(destination, file_name + '.gui'))


In [8]:
# make folder to store runtime files
if not os.path.exists('bin'):
    os.mkdir('bin')

### Declare magicode class

In [9]:
from tensorflow.keras.layers import Input, Dense, Dropout, \
                         RepeatVector, LSTM, concatenate, \
                         Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Sequential, Model, model_from_json
from tensorflow.keras.optimizers import RMSprop

from tensorflow.keras import *

class magicode:
    def __init__(self, input_shape, output_size, output_path):
        self.model = None
        self.name = 'magicode'
        self.input_shape = input_shape
        self.output_size = output_size
        self.output_path = output_path

        image_model = Sequential()
        image_model.add(Conv2D(32, (3, 3), padding='valid', activation='relu', input_shape=input_shape))
        image_model.add(Conv2D(32, (3, 3), padding='valid', activation='relu'))
        image_model.add(MaxPooling2D(pool_size=(2, 2)))
        image_model.add(Dropout(0.25))

        image_model.add(Conv2D(64, (3, 3), padding='valid', activation='relu'))
        image_model.add(Conv2D(64, (3, 3), padding='valid', activation='relu'))
        image_model.add(MaxPooling2D(pool_size=(2, 2)))
        image_model.add(Dropout(0.25))

        image_model.add(Conv2D(128, (3, 3), padding='valid', activation='relu'))
        image_model.add(Conv2D(128, (3, 3), padding='valid', activation='relu'))
        image_model.add(MaxPooling2D(pool_size=(2, 2)))
        image_model.add(Dropout(0.25))

        image_model.add(Flatten())
        image_model.add(Dense(1024, activation='relu'))
        image_model.add(Dropout(0.3))
        image_model.add(Dense(1024, activation='relu'))
        image_model.add(Dropout(0.3))

        image_model.add(RepeatVector(CONTEXT_LENGTH))

        visual_input = Input(shape=input_shape)
        encoded_image = image_model(visual_input)

        language_model = Sequential()
        language_model.add(LSTM(128, return_sequences=True, input_shape=(CONTEXT_LENGTH, output_size)))
        language_model.add(LSTM(128, return_sequences=True))

        textual_input = Input(shape=(CONTEXT_LENGTH, output_size))
        encoded_text = language_model(textual_input)

        decoder = concatenate([encoded_image, encoded_text])

        decoder = LSTM(512, return_sequences=True)(decoder)
        decoder = LSTM(512, return_sequences=False)(decoder)
        decoder = Dense(output_size, activation='softmax')(decoder)

        self.model = Model(inputs=[visual_input, textual_input], outputs=decoder)

        optimizer = RMSprop(lr=0.0001, clipvalue=1.0)
        self.model.compile(loss='categorical_crossentropy', optimizer=optimizer)

    def fit_generator(self, generator, steps_per_epoch):
        self.model.fit(generator, steps_per_epoch=steps_per_epoch, epochs=EPOCHS, verbose=1)
        self.save()

    def predict(self, image, partial_caption):
        return self.model.predict([image, partial_caption], verbose=0)[0]

    def save(self):
        model_json = self.model.to_json()
        with open(format(self.output_path, self.name, '.json'), "w") as json_file:
            json_file.write(model_json)
        self.model.save_weights(join(self.output_path, self.name, '.h5'))

    def load(self, name=""):
        output_name = self.name if name == "" else name
        with open(join(self.output_path, output_name, '.json'), "r") as json_file:
            loaded_model_json = json_file.read()
        self.model = model_from_json(loaded_model_json)
        self.model.load_weights(join(self.output_path, output_name, '.h5'))

### Train the model using a generator (to avoid having to fit all the data in memory)

In [10]:
import tensorflow as tf

np.random.seed(1234)

training_features = join('dataset', 'training_features')
output_path = join('bin')

dataset = Dataset()
dataset.load(training_features, generate_binary_sequences=True)
dataset.save_metadata(output_path)
dataset.voc.save(output_path)

gui_paths, img_paths = Dataset.load_paths_only(training_features)

input_shape = dataset.input_shape
output_size = dataset.output_size
steps_per_epoch = dataset.size / BATCH_SIZE
voc = Vocabulary()
voc.retrieve(output_path)

generator = Generator.data_generator(voc, gui_paths, img_paths, batch_size=BATCH_SIZE, generate_binary_sequences=True)

model = magicode(input_shape, output_size, output_path)

model.fit_generator(generator, steps_per_epoch=steps_per_epoch)

Loading data...
Generating sparse vectors...
Dataset size: 75255
Vocabulary size: 26
Input shape: (256, 256, 3)
Output size: 26
Parsing data...
Epoch 1/10
   2/1175 [..............................] - ETA: 3:32:04 - loss: 3.1206

KeyboardInterrupt: 

### Create the directories for storing screenshots to "decode" and the resulting code

In [13]:
# create directory to store images to be "decoded"
if not os.path.exists('screenshots_to_convert'):
    os.mkdir('screenshots_to_convert')
# create directory to store HTML code generated by "decoding" images in screenshots folder 
if not os.path.exists('generated_html'):
    os.mkdir('generated_html')

### Generate the code for provided screenshots - add files to the screenshots folder

In [ ]:
trained_weights_path = 'bin'
trained_model_name = 'magicode'
input_path = 'screenshots_to_convert'
output_path = 'generated_html'

meta_dataset = np.load(join(trained_weights_path, 'meta_dataset.npy'), allow_pickle=True)
input_shape = meta_dataset[0]
output_size = meta_dataset[1]

model = magicode(input_shape, output_size, trained_weights_path)
model.load(trained_model_name)

sampler = Sampler(trained_weights_path, input_shape, output_size, CONTEXT_LENGTH)

for f in os.listdir(input_path):
    if f.find('.png') != -1:
        evaluation_img = get_preprocessed_img(join(input_path,p), IMAGE_SIZE)

        file_name = f[:f.find('.png')]

        result, _ = sampler.predict_greedy(model, np.array([evaluation_img]))
        print('Result greedy: {}'.format(result))

        with open(join(output_path, file_name + '.gui'), 'w') as out_f:
            out_f.write(result.replace(START_TOKEN, '').replace(END_TOKEN, ''))

### Declare compiler

In [ ]:
import json
from classes.Node import *

class Compiler:
    def __init__(self, dsl_mapping_file_path):
        with open(dsl_mapping_file_path) as data_file:
            self.dsl_mapping = json.load(data_file)

        self.opening_tag = self.dsl_mapping['opening-tag']
        self.closing_tag = self.dsl_mapping['closing-tag']
        self.content_holder = self.opening_tag + self.closing_tag

        self.root = Node('body', None, self.content_holder)

    def compile(self, input_file_path, output_file_path, rendering_function=None):
        dsl_file = open(input_file_path)
        current_parent = self.root

        for token in dsl_file:
            token = token.replace(' ', '').replace('\n', '')

            if token.find(self.opening_tag) != -1:
                token = token.replace(self.opening_tag, '')

                element = Node(token, current_parent, self.content_holder)
                current_parent.add_child(element)
                current_parent = element
            elif token.find(self.closing_tag) != -1:
                current_parent = current_parent.parent
            else:
                tokens = token.split(',')
                for t in tokens:
                    element = Node(t, current_parent, self.content_holder)
                    current_parent.add_child(element)

        output_html = self.root.render(self.dsl_mapping, rendering_function=rendering_function)
        with open(output_file_path, 'w') as output_file:
            output_file.write(output_html)

### Compile the generated code

In [ ]:
FILL_WITH_RANDOM_TEXT = True
TEXT_PLACE_HOLDER = '[]'

dsl_path = join('compiler','assets','dsl-mappgin.json')
compiler = Compiler(dsl_path)

def render_content_with_text(key, value):
    text_inputs = ['input-text', 'input-password']
    control_inputs = ['input-checkbox', 'input-radio']
    if FILL_WITH_RANDOM_TEXT:
        if key.find('btn') != -1:
            value = value.replace(TEXT_PLACE_HOLDER, Utils.get_random_text())
        elif key.find('title') != -1:
            value = value.replace(TEXT_PLACE_HOLDER, Utils.get_random_text(length_text=5, space_number=0))
        elif key.find('text') != -1:
            value = value.replace(TEXT_PLACE_HOLDER,
                                  Utils.get_random_text(length_text=56, space_number=7, with_upper_case=False))
        elif any(text_input in key for text_input in text_inputs):
            value = value.replace(TEXT_PLACE_HOLDER, Utils.get_random_text(length_text=30, space_number=0))
        elif any(control_input in key for control_input in control_inputs):
            value = value.replace(TEXT_PLACE_HOLDER, Utils.get_random_text(length_text=10, space_number=0))
    return value

path = 'code'
generated_code_files = os.listdir(path)

for file in generated_code_files:
    file_uid = Path(file).stem
    input_file_path = join(path, file_uid, '.gui')
    output_file_path = join(path, file_uid, '.html')

    compiler.compile(input_file_path, output_file_path, rendering_function=render_content_with_text)